In [1]:
import pyodbc
from import_function import import_scoring_data, import_data 
from tto_pricing_features_2 import tto_pricing_features 
# from tto_pricing_features import tto_pricing_features
ct='New'
response='TOTAL_REVENUE_CY'
# TOTAL_REVENUE_CY,AGE_SPOUSE,	AGE_TAXPAYER,	AMOUNT_CHILD_CREDIT,	AMOUNT_EITC,	AMOUNT_HOPE_CREDIT,	AMOUNT_STUDENT_LOAN_INTEREST_DEDUCTION,	CHANNEL,	CUSTOMER_DEFINITION_ADJ,	ENTRY_PAGE_GROUP,	FED_FORM_TYPE,	FILING_STATUS,	FLAG_ITEMIZED_DEDUCTIONS,	FSCHA_FLAG,	IMPORT_TYPE,	MAX_FLAG,	MINDBENDER_FLAG,	MISC1099_FLAG,	NEAUTH_DEVICE_TYPE,	NON_CA_REFUND_TRANSFER_FLAG,	NUM_CARE_CONTACTS,	POST_TOTAL_VOTES,	PRE_TOTAL_VOTES,	PRODUCT_EDITION_DESCRIPTION,	PRODUCT_ROLLUP,	PRS_SCORE,	REFUND_TRANSFER_FLAG,	REJECT_COUNT,	REQUIRED_TAKE_FLAG,	START_DEVICE_TYPE,	STATE_ATTACH_COUNT,	student_taxpayer,	VAUTH_DEVICE_TYPE
#   and CUSTOMER_DEFINITION_ADJ='NEW TO TURBOTAX'
cont_features, bool_features, catag_features, cont_score_features, bool_score_features, catag_score_features=tto_pricing_features(ct) 
scoring_data = "SELECT A.* FROM (SELECT * FROM CTG_ANALYTICS_WS.SM_RETENTION_MODEL WHERE TAX_YEAR=2015 AND CUSTOMER_DEFINITION_ADJ='NEW TO TURBOTAX' )A inner join (SELECT DISTINCT CUSTOMER_KEY FROM CTG_ANALYTICS_WS.SM_RETENTION_MODEL WHERE TAX_YEAR=2015 AND TAX_DAY=120) B ON A.CUSTOMER_KEY=B.CUSTOMER_KEY limit 100" 
data = "SELECT TOTAL_REVENUE_CY, AGE_DEPENDENT_MAX,	AGE_SPOUSE,	AGE_TAXPAYER,	AMOUNT_CHILD_CARE_CREDIT,	AMOUNT_EDUCATION_CREDIT,	AMOUNT_EITC,	AMOUNT_HOPE_CREDIT,	AMOUNT_STUDENT_LOAN_INTEREST_DEDUCTION,	CHANNEL,	CUSTOMER_DEFINITION_ADJ,	DMA_AREA,	ENTRY_PAGE_GROUP,	FED_FORM_TYPE,	FILING_STATUS,	FIRST_COMPLETE_APP_TYPE,	FSCHA_FLAG,	FSCHD_FLAG,	FSCHE_FLAG,	IMPORT_TYPE,	LAST_STATUS,	MAX_FLAG,	MISC1099_FLAG,	NEAUTH_DEVICE_TYPE,	NON_CA_REFUND_TRANSFER_FLAG,	NUM_CARE_CONTACTS,	NUM_SCHC,	POST_TOTAL_VOTES,	PRE_TOTAL_VOTES,	PRODUCT_EDITION_DESCRIPTION,	PRS_SCORE,	REJECT_COUNT,	REQUIRED_TAKE_FLAG,	START_DEVICE_TYPE,	STATE_ATTACH_COUNT,	student_taxpayer,	VAUTH_DEVICE_TYPE,	REFUND_TRANSFER_FLAG,	SESSIONS_TO_COMPLETE FROM CTG_ANALYTICS_WS.SM_RETENTION_MODEL WHERE TAX_YEAR=2015 order by random() limit 2000000"
df_no_pca = import_data(data, cont_features, bool_features,response, catag_features)
# scoring_df=import_scoring_data(scoring_data, cont_score_features, bool_score_features,catag_score_features)
print df_no_pca

['TOTAL_REVENUE_CY', 'AGE_DEPENDENT_MAX', 'AGE_SPOUSE', 'AGE_TAXPAYER', 'AMOUNT_CHILD_CARE_CREDIT', 'AMOUNT_EDUCATION_CREDIT', 'AMOUNT_EITC', 'AMOUNT_HOPE_CREDIT', 'AMOUNT_STUDENT_LOAN_INTEREST_DEDUCTION', 'CHANNEL', 'CUSTOMER_DEFINITION_ADJ', 'DMA_AREA', 'ENTRY_PAGE_GROUP', 'FED_FORM_TYPE', 'FILING_STATUS', 'FIRST_COMPLETE_APP_TYPE', 'FSCHA_FLAG', 'FSCHD_FLAG', 'FSCHE_FLAG', 'IMPORT_TYPE', 'LAST_STATUS', 'MAX_FLAG', 'MISC1099_FLAG', 'NEAUTH_DEVICE_TYPE', 'NON_CA_REFUND_TRANSFER_FLAG', 'NUM_CARE_CONTACTS', 'NUM_SCHC', 'POST_TOTAL_VOTES', 'PRE_TOTAL_VOTES', 'PRODUCT_EDITION_DESCRIPTION', 'PRS_SCORE', 'REJECT_COUNT', 'REQUIRED_TAKE_FLAG', 'START_DEVICE_TYPE', 'STATE_ATTACH_COUNT', 'student_taxpayer', 'VAUTH_DEVICE_TYPE', 'REFUND_TRANSFER_FLAG', 'SESSIONS_TO_COMPLETE']
['AGE_DEPENDENT_MAX', 'AGE_SPOUSE', 'AGE_TAXPAYER', 'AMOUNT_CHILD_CARE_CREDIT', 'AMOUNT_EDUCATION_CREDIT', 'AMOUNT_EITC', 'AMOUNT_HOPE_CREDIT', 'AMOUNT_STUDENT_LOAN_INTEREST_DEDUCTION', 'FSCHA_FLAG', 'FSCHD_FLAG', 'FSCHE_FL

c:\python27\lib\site-packages\pandas\core\frame.py:2762: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


['AGE_DEPENDENT_MAX', 'AGE_SPOUSE', 'AGE_TAXPAYER', 'AMOUNT_CHILD_CARE_CREDIT', 'AMOUNT_EDUCATION_CREDIT', 'AMOUNT_EITC', 'AMOUNT_HOPE_CREDIT', 'AMOUNT_STUDENT_LOAN_INTEREST_DEDUCTION', 'FSCHA_FLAG', 'FSCHD_FLAG', 'FSCHE_FLAG', 'MAX_FLAG', 'MISC1099_FLAG', 'NON_CA_REFUND_TRANSFER_FLAG', 'NUM_CARE_CONTACTS', 'POST_TOTAL_VOTES', 'PRE_TOTAL_VOTES', 'REJECT_COUNT', 'REQUIRED_TAKE_FLAG', 'student_taxpayer', 'REFUND_TRANSFER_FLAG', 'SESSIONS_TO_COMPLETE', 'TOTAL_REVENUE_CY']


import_function.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_bool['paid_something'] = 0
import_function.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_bool['paid_something'][df[response] > 0]=1
c:\python27\lib\site-packages\pandas\core\generic.py:4485: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
c:\python27\lib\site-packages\ipykernel\__main__.py:12: SettingWithCopyWarning: 
A 

df_cont done
data_scaled done
just_dummies done
df_trans done
new feature list done
        FSCHA_FLAG FSCHD_FLAG FSCHE_FLAG MAX_FLAG MISC1099_FLAG  \
0             True      False      False    False         False   
1            False      False      False    False         False   
2            False      False      False    False         False   
3            False      False      False    False         False   
4             True      False      False    False         False   
5            False      False      False    False         False   
6            False      False      False    False         False   
7             True       True      False     True         False   
8            False      False      False    False         False   
9            False      False      False    False         False   
10           False      False      False    False         False   
11           False      False      False    False         False   
12           False      False      False    F

In [2]:
from bin_cont_features import bin_pca
j='True'
response='paid_something'
df_pca=df_no_pca
dummy_pca, length_dict = bin_pca(df_pca,response, j)
print dummy_pca


['<=0.5', '>0.5', 'between 0.5 and 3.5', '>=3.5']
['<=25.5', '<=22.5', 'between 22.5 and 25.5', '>25.5', 'between 25.5 and 29.5', '>=29.5']
['<=25.5', '<=21.5', 'between 21.5 and 25.5', '>25.5', 'between 25.5 and 30.5', '>=30.5']
['<=0.5', '>0.5', 'between 0.5 and 983.5', '>=983.5']
['<=923.5', '<=17.5', 'between 17.5 and 923.5', '>923.5', 'between 923.5 and 1458.5', '>=1458.5']
['<=0.5', '>0.5', 'between 0.5 and 503.5', '>=503.5']
['<=452.5', '<=28.5', 'between 28.5 and 452.5', '>452.5', 'between 452.5 and 1000.5', '>=1000.5']
['<=0.5', '>0.5', 'between 0.5 and 961.5', '>=961.5']
['<=0.5', '>0.5', 'between 0.5 and 1.5', '>=1.5']
['<=0.5', '>0.5', 'between 0.5 and 4.5', '>=4.5']
['<=0.5', '>0.5', 'between 0.5 and 1.5', '>=1.5']
['<=0.5', '>0.5', 'between 0.5 and 6.5', '>=6.5']
['<=2.5', '<=1.5', 'between 1.5 and 2.5', '>2.5', 'between 2.5 and 4.5', '>=4.5']
        FSCHA_FLAG FSCHD_FLAG FSCHE_FLAG MAX_FLAG MISC1099_FLAG  \
0             True      False      False    False         False

In [3]:
ct='New'
response='paid_something'
# print dummy_pca.dtypes
from feature_clustering import feature_clustering
i='pca'
df_no_pca=dummy_pca
df_no_pca, y, plsca= feature_clustering(df_no_pca,response, i)

print df_no_pca, y




(200000, 382)
{160: 65.438422380112854, 130: 65.638212131927517, 100: 65.724245254400415, 70: 65.924920991939914, 170: 65.411911294820129, 140: 65.553731442102119, 110: 65.783736927893301, 80: 65.827406081159126, 50: 66.023247695643065, 180: 65.377437679750969, 150: 65.554656411383618, 120: 65.710352928477192, 90: 65.777093416104719, 60: 65.923669918419805, 190: 65.170026885248419}
50
        AGE_TAXPAYER_pcl_'<=25.5' AGE_TAXPAYER_pcl_'<=21.5'  \
0                           False                     False   
1                           False                      True   
2                           False                      True   
3                           False                      True   
4                           False                     False   
5                           False                      True   
6                           False                      True   
7                           False                     False   
8                           False            

In [4]:
from clustering import obs_clustering
clust='False'
df=df_no_pca
print df.shape, list(df)

df, df_w_labels, df_w_clust=obs_clustering(df,response,clust)
# print df.shape, df_w_labels.shape, df_w_clust.shape

(2000000, 43) ["AGE_TAXPAYER_pcl_'<=25.5'", "AGE_TAXPAYER_pcl_'<=21.5'", u'START_DEVICE_TYPE_Phone', "AGE_SPOUSE_pcl_'between 22.5 and 25.5'", u'FED_FORM_TYPE_F1040A', 'PRS_SCORE_8.0', u'ENTRY_PAGE_GROUP_Microsite Pages', 'PRS_SCORE_7.0', 'student_taxpayer', u'CUSTOMER_DEFINITION_ADJ_NEW TO TURBOTAX', u'PRODUCT_EDITION_DESCRIPTION_TURBOTAX WEB PREMIER', u'ENTRY_PAGE_GROUP_Product and Pricing Pages', u'ENTRY_PAGE_GROUP_MyTurboTax', "AGE_TAXPAYER_pcl_'between 21.5 and 25.5'", u'CUSTOMER_DEFINITION_ADJ_PY SKIPPER', u'FIRST_COMPLETE_APP_TYPE_Mobile Web', 'MAX_FLAG', "AMOUNT_EITC_pcl_'>0.5'", u'NEAUTH_DEVICE_TYPE_Phone', u'FED_FORM_TYPE_F1040', u'DMA_AREA_New York', u'CHANNEL_Organic Search', u'CUSTOMER_DEFINITION_ADJ_LOYAL', 'PRS_SCORE_10.0', u'PRODUCT_EDITION_DESCRIPTION_TURBOTAX WEB PREMIER HOME & BUSINESS', "AMOUNT_STUDENT_LOAN_INTEREST_DEDUCTION_pcl_'>=961.5'", u'ENTRY_PAGE_GROUP_Campaign Landing Pages', "AMOUNT_HOPE_CREDIT_pcl_'<=28.5'", "AMOUNT_EDUCATION_CREDIT_pcl_'<=17.5'", 'MISC10

In [5]:
from pca_dim_red import pca_code
i='False'
response='paid_something'
df_no_pca=df

df_pca, fitted_pca = pca_code(df_no_pca,response,i)
print df_pca



        AGE_TAXPAYER_pcl_'<=25.5' AGE_TAXPAYER_pcl_'<=21.5'  \
0                           False                     False   
1                           False                      True   
2                           False                      True   
3                           False                      True   
4                           False                     False   
5                           False                      True   
6                           False                      True   
7                           False                     False   
8                           False                      True   
9                           False                      True   
10                          False                      True   
11                          False                     False   
12                          False                     False   
13                          False                      True   
14                          False                      

In [ ]:
from get_arrays import get_arrays
i='False'
j='False'
dummy_pca=df_pca
x, y = get_arrays(dummy_pca, response, i, j)
print x.shape
print x.shape
from sklearn.feature_selection import SelectPercentile, f_classif
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
selector = SelectPercentile(f_classif, percentile=0.5)
selector.fit(x, y)
print selector.pvalues_
df_pvalues=pd.DataFrame(selector.pvalues_,columns=['p_value'], index=list(x))
sig_features=df_pvalues[df_pvalues['p_value'] <=0.05]
sig_features.reset_index(['index'], inplace=True)
sig_features.drop(['p_value'], axis=1, inplace=True)
print sig_features


x=x[sig_features['index'].tolist()]
print x.shape


(2000000, 42)
(2000000, 42)
[  0.00000000e+000   0.00000000e+000   0.00000000e+000   0.00000000e+000
   0.00000000e+000   0.00000000e+000   0.00000000e+000   1.23330697e-074
   0.00000000e+000   0.00000000e+000   0.00000000e+000   0.00000000e+000
   0.00000000e+000   0.00000000e+000   0.00000000e+000   0.00000000e+000
   0.00000000e+000   3.43751211e-010   0.00000000e+000   0.00000000e+000
   2.60892020e-211   5.57192159e-172   0.00000000e+000   2.48249361e-002
   0.00000000e+000   0.00000000e+000   0.00000000e+000   0.00000000e+000
   0.00000000e+000   0.00000000e+000   1.52887944e-002   5.19128031e-046
   7.84633694e-082   8.26250555e-104   3.67173940e-158   1.13635099e-322
   0.00000000e+000   0.00000000e+000   0.00000000e+000   0.00000000e+000
   0.00000000e+000   0.00000000e+000]
                                                index
0                           AGE_TAXPAYER_pcl_'<=25.5'
1                           AGE_TAXPAYER_pcl_'<=21.5'
2                             START_DEVICE

In [ ]:
from model_algorithm import algorithm
print x.shape
models, name = algorithm(x, y, response)

c:\python27\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


(2000000, 42)
Decision_Tree 0.792046946735 0.717996829185 0.720149639099 0.715434478995 209645 84440 86269 219646
Random_Forest 0.793164709926 0.698115489597 0.715748621298 0.717670560821 216807 77278 92351 213564
                                                         sig
FED_FORM_TYPE_F1040                                 0.430561
NON_CA_REFUND_TRANSFER_FLAG                         0.138421
CUSTOMER_DEFINITION_ADJ_NEW TO TURBOTAX             0.092290
PRODUCT_EDITION_DESCRIPTION_TURBOTAX WEB PREMIER    0.052252
AGE_SPOUSE_pcl_'between 22.5 and 25.5'              0.043507
AGE_TAXPAYER_pcl_'between 21.5 and 25.5'            0.039793
AGE_TAXPAYER_pcl_'<=25.5'                           0.031197
FED_FORM_TYPE_F1040A                                0.031172
MAX_FLAG                                            0.021702
AGE_TAXPAYER_pcl_'<=21.5'                           0.016974
REQUIRED_TAKE_FLAG                                  0.015415
PRODUCT_EDITION_DESCRIPTION_TURBOTAX WEB PREMIE...  0.

c:\python27\lib\site-packages\sklearn\discriminant_analysis.py:695: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


QDA 0.738110546026 0.432389154496 0.549851546787 0.643451722045 251757 42863 173337 132043
Bagging 0.791791045663 0.708225817015 0.717418490249 0.716179299112 213344 81276 89102 216278
ERT 0.791782121735 0.708196345537 0.717401724598 0.716167957576 213346 81274 89111 216269


In [ ]:
import pandas as pd
import numpy as np
from predicting_code import transform_to_pca, bin_pca_score_set, get_scoring_arrays, score_set_feature_selection, predict,get_scoring_arrays
defection=pd.DataFrame()
i, j, k=['False','True','False' ]
df_trans_pca = transform_to_pca(scoring_df, fitted_pca, i)
scoring_df_trans = bin_pca_score_set(df_trans_pca, length_dict, j)

print scoring_df_trans.shape, x.shape
a=list(x)
b=list(scoring_df_trans)
final=list(set(b).difference(a))
scoring_df_trans.drop(final, axis=1, inplace=True)
print scoring_df_trans.shape
x.drop(list(scoring_df_trans), axis=1, inplace=True)
print x.shape
c=list(x)

m=scoring_df_trans.shape[0]     
n= x.shape[1]
print  scoring_df_trans.shape, x.shape

zeros_df=pd.DataFrame(np.zeros((m, n)), columns=c)



scoring_df_trans.reset_index(['CUSTOMER_KEY'], inplace=True)

final_scoring_df=pd.concat([zeros_df,scoring_df_trans ], axis=1)
final_scoring_df.set_index('CUSTOMER_KEY', inplace=True)
print final_scoring_df.shape

scoring_df_trans=final_scoring_df
x_score, index=score_set_feature_selection(scoring_df_trans,plsca, k)
x_score.set_index('CUSTOMER_KEY', inplace=True)
df_p=predict(models,name,x_score, index)
defection=pd.concat([df_p, defection], axis=1)
print defection.head()


In [ ]:
from data_intersection import scoring_data_intersection
data = scoring_data_intersection(df_no_pca, scoring_df)
# print data.head()
print df_cont